# Day 15: Beacon Exclusion Zone

[https://adventofcode.com/2022/day/15](https://adventofcode.com/2022/day/15)

## Description

### Part One

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable _sensors_ that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source _beacon_. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can _determine the position of a beacon precisely_; however, sensors can only lock on to the one beacon _closest to the sensor_ as measured by the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry). (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:

    Sensor at x=2, y=18: closest beacon is at x=-2, y=15
    Sensor at x=9, y=16: closest beacon is at x=10, y=16
    Sensor at x=13, y=2: closest beacon is at x=15, y=3
    Sensor at x=12, y=14: closest beacon is at x=10, y=16
    Sensor at x=10, y=20: closest beacon is at x=10, y=16
    Sensor at x=14, y=17: closest beacon is at x=10, y=16
    Sensor at x=8, y=7: closest beacon is at x=2, y=10
    Sensor at x=2, y=0: closest beacon is at x=2, y=10
    Sensor at x=0, y=11: closest beacon is at x=2, y=10
    Sensor at x=20, y=14: closest beacon is at x=25, y=17
    Sensor at x=17, y=20: closest beacon is at x=21, y=22
    Sensor at x=16, y=7: closest beacon is at x=15, y=3
    Sensor at x=14, y=3: closest beacon is at x=15, y=3
    Sensor at x=20, y=1: closest beacon is at x=15, y=3
    

So, consider the sensor at `2,18`; the closest beacon to it is at `-2,15`. For the sensor at `9,16`, the closest beacon to it is at `10,16`.

Drawing sensors as `S` and beacons as `B`, the above arrangement of sensors and beacons looks like this:

                   1    1    2    2
         0    5    0    5    0    5
     0 ....S.......................
     1 ......................S.....
     2 ...............S............
     3 ................SB..........
     4 ............................
     5 ............................
     6 ............................
     7 ..........S.......S.........
     8 ............................
     9 ............................
    10 ....B.......................
    11 ..S.........................
    12 ............................
    13 ............................
    14 ..............S.......S.....
    15 B...........................
    16 ...........SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....
    

This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at `8,7`:

                   1    1    2    2
         0    5    0    5    0    5
    -2 ..........#.................
    -1 .........###................
     0 ....S...#####...............
     1 .......#######........S.....
     2 ......#########S............
     3 .....###########SB..........
     4 ....#############...........
     5 ...###############..........
     6 ..#################.........
     7 .#########S#######S#........
     8 ..#################.........
     9 ...###############..........
    10 ....B############...........
    11 ..S..###########............
    12 ......#########.............
    13 .......#######..............
    14 ........#####.S.......S.....
    15 B........###................
    16 ..........#SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....
    

This sensor's closest beacon is at `2,10`, and so you know there are no beacons that close or closer (in any positions marked `#`).

None of the detected beacons seem to be producing the distress signal, so you'll need to <span title="&quot;When you have eliminated all which is impossible, then whatever remains, however improbable, must be where the missing beacon is.&quot; - Sherlock Holmes">work out</span> where the distress beacon is by working out where it _isn't_. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where `y=10`, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

                     1    1    2    2
           0    5    0    5    0    5
     9 ...#########################...
    10 ..####B######################..
    11 .###S#############.###########.
    

In this example, in the row where `y=10`, there are _`26`_ positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. _In the row where `y=2000000`, how many positions cannot contain a beacon?_


In [10]:
from pathlib import Path
from typing import Generator
import re

TEST = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3""".splitlines()

EXPECTED_1 = 26
EXPECTED_2 = None

DATA = Path("input/data15.txt").read_text().splitlines()


def parse(data: list[str]) -> Generator[tuple[int, int, int, int], None, None]:
    rows = []
    for line in data:
        if not line:
            continue
        match = re.match(
            "Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)",
            line,
        )
        # print(line, match.groups())
        sx, sy, bx, by = match.groups()
        yield int(sx), int(sy), int(bx), int(by)


def excluded_at_y(y: int, sx: int, sy: int, dist: int) -> set[int]:
    # print(sx, sy, dist, y, abs(y-sy), dist - abs(y-sy))
    dist -= abs(y - sy)
    if dist > 0:
        return {x for x in range(sx - dist, sx + dist + 1)}
    return set()


def score_1(data: list[str], y: int) -> int:
    excludes: set[int] = set()
    beacons = set()
    for sx, sy, bx, by in parse(data):
        if by == y:
            beacons.add((bx, by))
        dist = abs(sx - bx) + abs(sy - by)
        excl = excluded_at_y(y, sx, sy, dist)
        excludes |= excl

    return len(excludes) - len(beacons)


test_score = score_1(TEST, y=10)
print("test", test_score)
assert test_score == EXPECTED_1
print("part 1", score_1(DATA, y=2000000)) # 5367037


test 26
part 1 5367037


<div class="alert alert-info">Part 1 was no problem.</div>


### Part Two

Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have `x` and `y` coordinates each no lower than `0` and no larger than `4000000`.

To isolate the distress beacon's signal, you need to determine its _tuning frequency_, which can be found by multiplying its `x` coordinate by `4000000` and then adding its `y` coordinate.

In the example above, the search space is smaller: instead, the `x` and `y` coordinates can each be at most `20`. With this reduced search area, there is only a single position that could have a beacon: `x=14, y=11`. The tuning frequency for this distress beacon is _`56000011`_.

Find the only possible position for the distress beacon. _What is its tuning frequency?_


<div class="alert alert-info">Part 2 largely defeated my covid befuddled brain. I tried to work out where the zone edges intersected but couldn't get that code to work so got the solution by simply brute-forcing along the edges. A few days later I returned to the code so here's what I really meant to write the first time.</div>

In [11]:
from pathlib import Path
from typing import Generator
import re

TEST = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3""".splitlines()

EXPECTED_1 = 26
EXPECTED_2 = None

DATA = Path("input/data15.txt").read_text().splitlines()


def parse(data: list[str]) -> Generator[tuple[int, int, int, int], None, None]:
    rows = []
    for line in data:
        if not line:
            continue
        match = re.match(
            "Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)",
            line,
        )
        # print(line, match.groups())
        sx, sy, bx, by = match.groups()
        yield int(sx), int(sy), int(bx), int(by)


def mdist(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])


def show(p1, r1, p2, r2, maybe=()):
    print(p1, p2)
    p1x, p1y = p1
    p2x, p2y = p2
    lowy = min(0, min(p1y - r1, p2y - r2) - 1)
    hiy = max(p1y + r1, p2y + r2) + 1
    minx = min(0, min(p1x - r1, p2x - r2) - 1)
    hix = max(p1x + r1, p2x + r2) + 1
    for y in range(hiy + 2, lowy - 1, -1):
        print(
            "".join(
                "X"
                if (x, y) in maybe
                else "S"
                if (x, y) in {p1, p2}
                else "#"
                if mdist(p1, (x, y)) <= r1 or mdist(p2, (x, y)) <= r2
                else "."
                for x in range(minx, hix + 1)
            )
        )


def intersect_diagonals(m: int, b: int, n: int, c: int) -> tuple[int, int]:
    """Intersect two diagonals.
    {m,n} = {1,-1}
    switch if needed so m=1
        x+b = -x+c => 2x = c-b, 2y = 2x+2b
    Returns coordinates doubles to keep them int.
    """
    if m == -1:
        b, c = c, b
    two_x = c - b
    two_y = two_x + 2 * b
    return (two_x, two_y)


def candidates(
    px: int, py: int, pr: int, qx: int, qy: int, qr: int, limit: int = 4_000_000
):
    """Given two sensors that may overlap, find points exactly 1 step out of range
    of both.
    Each sensor has 4 edges:
    A: y = x + py - (px - pr - 1),
    B: y = -x + py + (px + pr + 1)
    C: y = x + py - (px + pr + 1)
    D: y = -x + py + (px - pr - 1)

    Find all the intersections. If not exact integers use the surrounding points
    """
    p_edges = [
        (1, py - (px - pr - 1)),
        (-1, py + (px + pr + 1)),
        (1, py - (px + pr + 1)),
        (-1, py + (px - pr - 1)),
    ]
    q_edges = [
        (1, qy - (qx - qr - 1)),
        (-1, qy + (qx + qr + 1)),
        (1, qy - (qx + qr + 1)),
        (-1, qy + (qx - qr - 1)),
    ]

    candidates = {
        intersect_diagonals(m, b, n, c)
        for m, b in p_edges
        for n, c in q_edges
        if m != n
    }

    # divide coords by 2. When not an int replace with surrounding points.
    c2 = []
    for two_x, two_y in candidates:
        c2.extend(
            [
                (x, y)
                for x in {two_x // 2, (two_x + 1) // 2}
                for y in {two_y // 2, (two_y + 1) // 2}
            ]
        )

    # Only keep candidates within the bounding boxes
    candidates = [
        (x, y)
        for x, y in c2
        if px - pr - 1 <= x <= px + pr + 1
        and py - pr - 1 <= y <= py + pr + 1
        and qx - qr - 1 <= x <= qx + qr + 1
        and qy - qr - 1 <= y <= qy + qr + 1
        and 0 <= x <= limit
        and 0 <= y <= limit
    ]

    # filter out points in range introduced by division
    candidates = [
        xy for xy in candidates if mdist(xy, (px, py)) > pr and mdist(xy, (qx, qy)) > qr
    ]

    return candidates


def debug(c1, r1, c2, r2):
    maybe = set(candidates(c1[0], c1[1], r1, c2[0], c2[1], r2))
    print(maybe)
    show(c1, r1, c2, r2, maybe)


def score_2(data: list[str], maxy: int) -> int:
    beacons = set()
    sensors = {}
    maybe = set()
    for sx, sy, bx, by in parse(data):
        beacons.add((bx, by))
        dist = abs(sx - bx) + abs(sy - by)
        sensors[sx, sy] = dist

    maybe = []
    for c1, pr in sensors.items():
        px, py = c1
        for c2, qr in sensors.items():
            if c2 <= c1:
                continue
            qx, qy = c2
            maybe.extend(candidates(px, py, pr, qx, qy, qr, maxy))

    print(f"{len(sensors)} sensors, {len(maybe)} candidates")

    for p in maybe:
        if all(mdist(p, s) > r for s, r in sensors.items()):
            print("Found position", p)
            return p[0] * 4_000_000 + p[1]
    raise RuntimeError("blah!")


EXPECTED_2 = 56000011

if EXPECTED_2 is not None:
    test_score = score_2(TEST, maxy=20)
    print("test", test_score)
    assert test_score == EXPECTED_2
    print("part 2", score_2(DATA, maxy=4_000_000))


14 sensors, 109 candidates
Found position (14, 11)
test 56000011
35 sensors, 285 candidates
Found position (2978645, 3249288)
part 2 11914583249288


<div class="alert alert-info">The answer must be bounded on 4 sides by zones so it must be a place where the zone edges intersect. With this version of the code I find the lines that run just outside the four edges of each zone and taking the zones in pairs find where those lines intersect (if anywhere). Note that because of the integer coordinates some intersections actually result in a pair of candidate points. Then I just checked each of the 285 intersections to find which one is outside all zones.</div>

In [12]:
debug((8, 7), 9, (13, 2), 3)

{(15, 4), (11, 0)}
(8, 7) (13, 2)
.....................
.....................
.....................
..........#..........
.........###.........
........#####........
.......#######.......
......#########......
.....###########.....
....#############....
...###############...
..#################..
.#########S#########.
..#################..
...###############...
....#############X...
.....#############...
......#########S###..
.......###########...
........#####X###....
.........###...#.....
..........#..........
.....................


In [13]:
debug((8, 2), 9, (13, 6), 3)

{(16, 4), (11, 9), (16, 5), (12, 9)}
(8, 2) (13, 6)
.....................
.....................
.....................
..........#..........
.........###.........
........#####XX#.....
.......##########....
......############...
.....##########S###..
....##############X..
...###############X..
..#################..
.#########S#########.
..#################..
...###############...
....#############....
.....###########.....
......#########......
.......#######.......
........#####........
.........###.........
..........#..........
.....................
